#### CSCE 489 :: Recommender Systems :: Texas A&M University :: Spring 2021


# Homework 2: Collaborative Filtering

### 100 points [10% of your final grade]

- **Due Friday, February 26 by 11:59pm**

*Goals of this homework:* The objective of this homework is to turn the theory of collaborative filtering into practice, and compare the results with your naive non-personalized recommendation from Homework 1 to see how personalized collaborative filtering algorithms provide better recommendations for both the rating prediction task (with explicit feedback) and the ranking task (with implicit feedback).

*Submission instructions (Canvas):* To submit your homework, rename this notebook as `UIN_hw2.ipynb`. For example, if your UIN is `123456789`, then your homework submission would be `123456789_hw2.ipynb`. Submit this notebook via Canvas. Your notebook should be completely self-contained, with the results visible in the notebook. We should not have to run any code from the command line, nor should we have to run your code within the notebook (though we reserve the right to do so). So please run all the cells for us, and then submit. 

*Late policy:* You may use up to three of your late days on this assignment. No homeworks will be accepted after March 1 11:59pm.

## Collaboration Declaration:

***You must add all of your collaboration declarations here. Who did you talk to about this assignment? What web resources did you use? Etc.***

For example:
* Part 1b: I consulted the Teams channel on how to build the jaccard matrix and how to best deal with missing ratings
* referenced the NumPy api docs quite a lot since I don't have any experience with it
* Part 1c - I consulted this stack overflow thread to help calculate cosine similarity 
https://stackoverflow.com/questions/41905029/create-cosine-similarity-matrix-numpy
* I needed help figuring out how to get the top 10 nearest neighbors so i consulted this stack overflow thread on partitioning
https://stackoverflow.com/questions/6910641/how-do-i-get-indices-of-n-maximum-values-in-a-numpy-array



# Part 1. Recommendations with User Ratings (Explicit Feedback) (60 points total)

In this first part, similar to Part 1 in our Homework 1, we still focus on the rating prediction recommendation task with explicit feedback. But in this part, you will need to build **personalized** models instead of non-personalized models as in Homework 1. You will also evaluate your personalized models to compare them with the non-personalized one in Homework 1.

For this part, we will:

* load and process the MovieLens 1M dataset, 
* build a baseline estimation model,
* build a user-user collaborative filtering model,
* improve the user-user collaborative filtering model, and
* evaluate and compare these different models.

Before you start to build your recommendation models. First, we need to load and preprocess the experiment dataset. We still use the MovieLens 1M data from https://grouplens.org/datasets/movielens/1m/ in this homework, and use the same loading and preprocessing process as Homework 1 Part 1a. The code has been provided in the next cell, and you need to run it. The resulting data variables are the same as those in Homework 1: train_mat is the numpy array variable for training data of size (#users, #items) with non-zero entries representing user-item ratings, and zero entries representing unknown user-item ratings; and test_mat is the numpy array variable for testing data of size (#users, #items).

In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix

data_df = pd.read_csv('./ratings.dat', sep='::', names=["UserID", "MovieID", "Rating", "Timestamp"], engine='python')

# First, generate dictionaries for mapping old id to new id for users and movies
unique_MovieID = data_df['MovieID'].unique()
unique_UserID = data_df['UserID'].unique()
j = 0
user_old2new_id_dict = dict()
for u in unique_UserID:
    user_old2new_id_dict[u] = j
    j += 1
j = 0
movie_old2new_id_dict = dict()
for i in unique_MovieID:
    movie_old2new_id_dict[i] = j
    j += 1
    
# Then, use the generated dictionaries to reindex UserID and MovieID in the data_df
user_list = data_df['UserID'].values
movie_list = data_df['MovieID'].values
for j in range(len(data_df)):
    user_list[j] = user_old2new_id_dict[user_list[j]]
    movie_list[j] = movie_old2new_id_dict[movie_list[j]]
data_df['UserID'] = user_list
data_df['movieID'] = movie_list

# generate train_df with 70% samples and test_df with 30% samples, and there should have no overlap between them.
train_index = np.random.random(len(data_df)) <= 0.7
train_df = data_df[train_index]
test_df = data_df[~train_index]

# generate train_mat and test_mat
num_user = len(data_df['UserID'].unique())
num_movie = len(data_df['MovieID'].unique())


train_mat = coo_matrix((train_df['Rating'].values, (train_df['UserID'].values, train_df['MovieID'].values)), shape=(num_user, num_movie)).astype(float).toarray()
test_mat = coo_matrix((test_df['Rating'].values, (test_df['UserID'].values, test_df['MovieID'].values)), shape=(num_user, num_movie)).astype(float).toarray()

## Part 1a: Build the Baseline Estimation Model (20 points)

First, let's implement a simple personalized recommendation model -- the baseline estimate -- introduced in class: $b_{u,i}=\mu+b_i+b_u$, where $\mu$ is the overall mean rating for all items, $b_u$ = average rating of user $u-\mu$, $b_i$ = average rating of item $i-\mu$. Store your prediction as a numpy array variable 'prediction_mat' of size (#users, #movies) with each entry showing the predicted rating for the corresponding user-movie pair.

* Hint: for users who do not have ratings in train_mat, set $b_u=0$ for them; and for movies which do not have ratings in train_mat, set $b_i=0$ for them

In [2]:
# calculate the prediction_mat by the baseline estimation recommendation algorithm
# Your Code Here...

def get_baseline_estimate(train_mat):

    prediction_mat = train_mat.copy()
    prediction_mat[prediction_mat == 0] = np.nan
    mu = np.nanmean(prediction_mat)

    num_rows = prediction_mat.shape[0]
    num_columns = prediction_mat.shape[1]

    # looping through each user to get it's bi
    curr_user_mean = 0.0
    bu = []
    for i in range(num_rows):
        curr_user_mean = np.nanmean(prediction_mat[i,:])
        bu.append(curr_user_mean - mu )

    # looping through each column to get movie rating deviation
    curr_movie_mean = 0.0
    bi = []
    for i in range(num_columns):
        curr_movie_mean = np.nanmean(prediction_mat[:,i])
        bi.append(curr_movie_mean - mu)



    bu = np.nan_to_num(bu)
    bi = np.nan_to_num(bi)

    for i in range(num_rows):
        for j in range(num_columns):
            prediction_mat[i][j] = bu[i] + bi[j] + mu
    
    prediction_mat = np.nan_to_num(prediction_mat)
    return prediction_mat


prediction_mat = get_baseline_estimate(train_mat)
print(prediction_mat)


<ipython-input-2-9db63bc87a43>:24: RuntimeWarning: Mean of empty slice
  curr_movie_mean = np.nanmean(prediction_mat[:,i])


[[4.9181489  3.98827012 4.70249492 ... 1.53918396 5.53918396 4.53918396]
 [4.57715656 3.64727778 4.36150258 ... 1.19819162 5.19819162 4.19819162]
 [4.71360344 3.78372466 4.49794946 ... 1.3346385  5.3346385  4.3346385 ]
 ...
 [4.96360344 4.03372466 4.74794946 ... 1.5846385  5.5846385  4.5846385 ]
 [4.63693678 3.707058   4.4212828  ... 1.25797184 5.25797184 4.25797184]
 [4.42036774 3.49048896 4.20471376 ... 1.0414028  5.0414028  4.0414028 ]]


Now, with this prediction_mat based on the baseline estimate, let's use the same RMSE metric as you used in Homework 1 Part 1c to evaluate the quality of the baseline estimate model. Please print out the RMSE of your prediction_mat using test_mat in the next cell.

In [3]:
# calculate and print out the RMSE for your prediction_df and the test_df
# Your Code Here...
indicator_mat = (test_mat > 0).astype(float)
test_rmse = (np.sum(((prediction_mat - test_mat) * indicator_mat) ** 2) / np.sum(indicator_mat)) ** 0.5
print(test_rmse)

0.9372309125262424


Do you observe an increase or decrease of the RMSE compared with the result you got from Homework 1 Part 1c with a non-personalized recommendation model? What do you think leads to this difference?

I saw a slight decrease compared to hw 1, meaning the prediciton is more accurate. This is expected since people are getting a personalized recommendation. This normalizes someone's rating and takes into account people who give all 4 or 5 star ratings vs. people who tend to give lower ratings for a movie. Taking into account the user's and movie's average mean in the baseline estimate gives us a more realistic predicted rating.

## Part 1b: User-user Collaborative Filtering with Jaccard Similarity (30 points)

In this part, you need to build a user-user collaborative filtering recommendation model with **Jaccard similarity** to predict user-movie ratings. 

The prediction of the score for a user-item pair $(u,i)$ should use the formulation: $p_{u,i}=\bar{r}_u+\frac{\sum_{u^\prime\in N}s(u,u^\prime)(r_{u^\prime,i}-\bar{r}_{u^\prime})}{\sum_{u^\prime\in N}|s(u, u^\prime)|}$ as introduced in class, where $s(u, u^\prime)$ is the Jaccard similarity. We set the size of $N$ as 10.

In the next cell, you need to write your code to implement this algorithm, and generate a numpy array variable named 'prediction_mat' of size (#user, #movie) with each entry showing the predicted rating for the corresponding user-movie pair.

* Hint: when you find the nearest neighbor set $N$ of a user $u$, do not include user $u$ in $N$  


In [4]:
# calculate the prediction_mat by your user-user collaborative filtering recommendation algorithm
# Your Code Here...
indicator_mat = (train_mat > 0).astype(float)

# gets array of the # of ratings for each user
num_ratings_per_user = np.sum(indicator_mat, axis=1,keepdims=True)

# returns array of the # of movies they both rated
numerator = np.matmul(indicator_mat, indicator_mat.T) # size = (#users, #users)

# get the # of movies both users both rated in total
denominator = num_ratings_per_user + num_ratings_per_user.T - numerator # size = (#users, #users)


#set 0 to be 1 to avoid error in division
denominator[denominator == 0] = 1

jaccard_matrix = numerator / denominator

#print (jaccard_matrix)

num_users = jaccard_matrix.shape[0]
kNearestNeighbors = []
for i in range(num_users):
    curr_users_neighbors = np.argpartition(jaccard_matrix[i], -11)[-11:] # getting 11 bc it will include the curr user
    curr_users_neighbors = curr_users_neighbors[curr_users_neighbors != i] # removing user himself from neighbors
    kNearestNeighbors.append(curr_users_neighbors)


print (kNearestNeighbors[0])
print (kNearestNeighbors[1])
# print(jaccard_matrix[0])
prediction_mat = train_mat.copy()
nan_prediction_mat = train_mat.copy() # creating a copy so it's easier to do mean calculations
nan_prediction_mat[nan_prediction_mat == 0] = np.nan
      
for i in range(num_users):
    curr_users_mean = np.nanmean(nan_prediction_mat[i])
    numerator = 0
    denom = 0

    for j in range(10):
        curr_neighbor = kNearestNeighbors[i][j]
        curr_neighbor_rating_vec = prediction_mat[curr_neighbor] # gets row of neighbor's ratinggi

        nan_curr_neighbor_rating_vec = nan_prediction_mat[curr_neighbor]
        curr_neighbors_mean = np.nanmean(curr_neighbor_rating_vec)
        if(curr_neighbors_mean == 0):
            continue
        similarity = jaccard_matrix[i][kNearestNeighbors[i][j]] # getting the similarity of each of the neighbors
        
        curr_neighbor_rating_vec[curr_neighbor_rating_vec == 0] = curr_users_mean
        rating_dev = curr_neighbor_rating_vec - curr_neighbors_mean
        rating_dev *= indicator_mat[curr_neighbor]
        numerator += similarity * rating_dev # sets instances where neighbor didn't give a rating to zero
        denom += similarity
    
    prediction_mat[i] = curr_users_mean +  (numerator / denom)
    
    
        


[4299 5342 1656 5189   79 5835  478 2500 1480 2649]
[3587 4140   12  809 3994   94  102  299 2813 4600]


Please print out the RMSE of your prediction_mat using test_mat in the next cell.

In [5]:
# calculate and print out the RMSE for your prediction_df and the test_df
# Your Code Here...
indicator_mat = (test_mat > 0).astype(float)
test_rmse = (np.sum(((prediction_mat - test_mat) * indicator_mat) ** 2) / np.sum(indicator_mat)) ** 0.5
print(test_rmse)

1.0188813316801908


Comparing the RMSE results of this user-user collaborative filtering, the baseline estimate algorithm, and your non-personalized recommendation model from Homework 1 Part 1c, what do you observe? Is the user-user collaborative filtering the one producing the best performance? What reasons do you think can explain what you observe?

It's slightly worse than the baseline estimate and slightly worse than the non-personalized recommendation model in HW1 Part c. I don't think it's the nature of user-user collaborative filtering that is making this worse necessarily, but it's because the jaccard matrix doesn't take into account actual ratings when determining how similar two users are. So it seems that non-personalized and the baseline recommenders are better than a personalized recommender when the similarity rating between user is not strong enough.

## Part 1c: Improve the Collaborative Filtering Model (20 points)

Next, you need to try your best to improve the collaborative filtering model so that we can improve our RMSE! This is open-ended, so feel free to try whatever collaborative filtering tricks you like. We talked about several in class, plus you can find more in the readings. As a starting point, here are some possibilities:

* Try the cosine similarity or pearson similarity measurement instead of Jaccard
* Try item-item collaborative filtering instead of user-user CF
* Try to include the baseline estimation model in your collaborative filtering model

Besides these, you can also figure out your own solutions how to improve the performance. Write your code in the next cell and print out the RMSE of your new model.

In [5]:
# implement your improved model and print out the RMSE
# Your Code Here...
# i'll be trying user-user cosine similarity measurement instead of jaccard

##### converting current ratings using baseline estimate
prediction_mat = train_mat.copy()

baseline_estimate = get_baseline_estimate(train_mat)

prediction_mat = train_mat.copy()
num_movies = prediction_mat.shape[1]

### calculating cosine similarity matrix
numerator = prediction_mat @ prediction_mat.T
norm = (prediction_mat * prediction_mat).sum(axis=1, keepdims=True) ** 0.5

cosine_sim_mat = numerator / norm /  norm.T



num_users = cosine_sim_mat.shape[0]
kNearestNeighbors = []
for i in range(num_users):
    curr_users_neighbors = np.argpartition(cosine_sim_mat[i], -11)[-11:] # getting 11 bc it will include the curr user
    curr_users_neighbors = curr_users_neighbors[curr_users_neighbors != i] # removing user himself from neighbors
    kNearestNeighbors.append(curr_users_neighbors)


prediction_mat = train_mat.copy()
nan_prediction_mat = train_mat.copy() # creating a copy so it's easier to do mean calculations
nan_prediction_mat[nan_prediction_mat == 0] = np.nan
    
indicator_mat = (train_mat > 0).astype(float)
for i in range(num_users):
    curr_users_mean = np.nanmean(nan_prediction_mat[i])
    numerator = 0
    denom = 0

    for j in range(10):
        curr_neighbor = kNearestNeighbors[i][j]
        curr_neighbor_rating_vec = prediction_mat[curr_neighbor] # gets row of neighbor's ratinggi

        nan_curr_neighbor_rating_vec = nan_prediction_mat[curr_neighbor]
        curr_neighbors_mean = np.nanmean(curr_neighbor_rating_vec)
        if(curr_neighbors_mean == 0):
            continue
        similarity = cosine_sim_mat[i][kNearestNeighbors[i][j]] # getting the similarity of each of the neighbors
        
        curr_neighbor_rating_vec[curr_neighbor_rating_vec == 0] = curr_users_mean
        rating_dev = curr_neighbor_rating_vec - curr_neighbors_mean
        rating_dev *= indicator_mat[curr_neighbor]
        numerator += similarity * rating_dev # sets instances where neighbor didn't give a rating to zero
        #numerator *= indicator_mat[curr_neighbor]
        denom += similarity
    
    prediction_mat[i] = curr_users_mean +  (numerator / denom)

#print(prediction_mat[0])
print(prediction_mat)


indicator_mat = (test_mat > 0).astype(float)
test_rmse = (np.sum(((prediction_mat - test_mat) * indicator_mat) ** 2) / np.sum(indicator_mat)) ** 0.5
print(test_rmse)

<ipython-input-2-9db63bc87a43>:24: RuntimeWarning: Mean of empty slice
  curr_movie_mean = np.nanmean(prediction_mat[:,i])


[[6.50808477 5.04574126 5.58341726 ... 4.12121212 4.12121212 4.12121212]
 [5.27570171 4.14170901 4.51913205 ... 3.78021978 3.78021978 3.78021978]
 [3.91666667 3.91666667 3.91666667 ... 3.91666667 3.91666667 3.91666667]
 ...
 [4.19625239 4.16666667 4.17283385 ... 4.16666667 4.16666667 4.16666667]
 [4.03147416 3.84049487 4.08499082 ... 3.84       3.84       3.84      ]
 [4.03330768 3.68012085 3.89105204 ... 3.62343096 3.62343096 3.62343096]]
1.0081794418705394


And please briefly explain what your new model does to improve the performance.

My model uses the cosine similarity measurement instead of jaccard to improve the performance. This factors in the actual ratings when calculating similarity, whereas jaccard looks only at whether or not two users have rated the same items, discarding the rating given.


# Part 2. Recommendations with implicit feedback (40 points total)

Now we turn to study how collaborative filtering algorithms work for recommendations with implicit feedback. The overall pipeline is the same as in Part 2 of Homework 1. But now you need to implement a user-user collaborative filtering algorithm for recommendation.

In this part, you will use the same MovieLens 1M dataset, and:
* write the code to implement a user-user collaborative filtering algorithm,
* and evaluate your recommender.

Before the algorithm implementation, we need to first transfer the explicit datasets you already generated to implicit ones. The process is the same as Part 2a in Homework 1, and the code has been provided. You need to run the code in the next cell.

In [6]:
train_mat = (train_mat > 0).astype(float)
test_mat = (test_mat > 0).astype(float)

Then, you need to write code to implement a user-user collaborative filtering algorithm with **cosine similarity**. Although we have not discussed in class how to use collaborative filtering for implicit feedback, it is quite straightforward. 

* We first need to calculate the cosine similarity between users based on the binary feedback vectors of users; 
* Then, for a specific user $u$, we predict a preference vector for the user $u$ by weighted averaging the binary feedback vectors of N users who are most similar to $u$;
* And last, rank the movies based on the predicted preference vector of the user $u$ as recommendations. 

The predicted preference score from user $u$ to movie $i$ can be calculated as: $p_{u,i}=\frac{\sum_{u^\prime\in N}s(u,u^\prime)r_{u^\prime,i}}{\sum_{u^\prime\in N}|s(u,u^\prime)|}$, where $s(u,u^\prime)$ is the cosine similarity, and we set the size of $N$ as 10.

In the next cell, write your code to generate the ranked lists of movies by this user-user collaborative filtering recommendation algorithm for every user, store the result in a numpy array of size (#user, 50), where entry (u, k) represents the movie id that is ranked at position k in the recommendation list to user u. 

* Hint: for a user $u$, the movies user $u$ liked in the train_mat should be excluded in the top 50 recommendation list

In [7]:
# Your Code Here...
user_num = test_mat.shape[0]
item_num = test_mat.shape[1]

print(train_mat)
popularity = np.sum(train_mat, axis=0)

prediction_mat = train_mat.copy()

numerator = prediction_mat @ prediction_mat.T
norm = (prediction_mat * prediction_mat).sum(axis=1, keepdims=True) ** 0.5


cosine_mat = numerator / norm /  norm.T
#print(similarities)



num_users = cosine_mat.shape[0]
kNearestNeighbors = []
for i in range(num_users):
    curr_users_neighbors = np.argpartition(cosine_mat[i], -11)[-11:] # getting 11 bc it will include the curr user
    curr_users_neighbors = curr_users_neighbors[curr_users_neighbors != i] # removing user himself from neighbors
    kNearestNeighbors.append(curr_users_neighbors)
    
# getting indices of movies a user has already seen
user_train_like = []
for u in range(user_num):
    user_train_like.append(np.where(train_mat[u, :] > 0)[0])



prediction_mat = train_mat.copy()
nan_prediction_mat = train_mat.copy() # creating a copy so it's easier to do mean calculations
nan_prediction_mat[nan_prediction_mat == 0] = np.nan
    
indicator_mat = (train_mat > 0).astype(float)
for i in range(num_users):
    curr_users_mean = np.nanmean(nan_prediction_mat[i])
    numerator = 0
    denom = 0

    for j in range(10):
        curr_neighbor = kNearestNeighbors[i][j]
        curr_neighbor_rating_vec = prediction_mat[curr_neighbor] # gets row of neighbor's ratinggi

        nan_curr_neighbor_rating_vec = nan_prediction_mat[curr_neighbor]
        curr_neighbors_mean = np.nanmean(curr_neighbor_rating_vec)
        if(curr_neighbors_mean == 0):
            continue
        similarity = cosine_mat[i][kNearestNeighbors[i][j]] # getting the similarity of each of the neighbors
        
        curr_neighbor_rating_vec[curr_neighbor_rating_vec == 0] = curr_users_mean
        rating_dev = curr_neighbor_rating_vec
        rating_dev *= indicator_mat[curr_neighbor]  # sets instances where neighbor didn't give a rating to zero
        numerator += similarity * rating_dev
        denom += similarity
    
    prediction_mat[i] = (numerator / denom)

    
popularity = np.sum(prediction_mat, axis=0)
recommendation = []
for u in range(user_num):
    train_like = user_train_like[u]
    scores = popularity.copy()
    scores[train_like] = -1
    top50_iid = np.argpartition(scores, -50)[-50:]
    top50_iid = top50_iid[np.argsort(scores[top50_iid])[-1::-1]]
    recommendation.append(top50_iid)
recommendation = np.array(recommendation)

print('top5 movies:' + str(recommendation[0, :5]))
print('top5 popularity:' + str(popularity[recommendation[0, :5]]))


[[1. 1. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
top5 movies:[104 124  97 113  64]
top5 popularity:[1508.2384193  1308.77635792 1242.67568385 1240.04693394 1185.80499485]


Last, you need to evaluate your user-user collaborative filtering algorithm by the held-out testing dataset test_mat for each user. Here, we use the same metrics recall@k and precision@k we used in Part 2c of Homework 1. So you can use the same code here to calculate recall@k and precision@k for k=5, 20, 50 for each user, i.e., six metrics for every user. And please print out the average over all users for these six metrics.

In [8]:
# Calculate recall@k, precision@k with k=5, 20, 50 and print out the average over all users for these 6 metrics.
# Your Code Here...

user_test_like = []
for u in range(user_num):
    user_test_like.append(np.where(test_mat[u, :] > 0)[0])
    
recalls = np.zeros(3)
precisions = np.zeros(3)
user_count = 0.

for u in range(user_num):
    test_like = user_test_like[u]
    test_like_num = len(test_like)
    if test_like_num == 0:
        continue
    rec = recommendation[u, :]
    hits = np.zeros(3)
    for k in range(50):
        if rec[k] in test_like:
            if k < 50:
                hits[2] += 1
                if k < 20:
                    hits[1] += 1
                    if k < 5:
                        hits[0] += 1
    recalls[0] += (hits[0] / test_like_num)
    recalls[1] += (hits[1] / test_like_num)
    recalls[2] += (hits[2] / test_like_num)
    precisions[0] += (hits[0] / 5.)
    precisions[1] += (hits[1] / 20.)
    precisions[2] += (hits[2] / 50.)
    user_count += 1

recalls /= user_count
precisions /= user_count

print('recall@5\t[%.6f],\t||\t recall@20\t[%.6f],\t||\t recall@50\t[%.6f]' % (recalls[0], recalls[1], recalls[2]))
print('precision@5\t[%.6f],\t||\t precision@20\t[%.6f],\t||\t precision@50\t[%.6f]' % (precisions[0], precisions[1], precisions[2]))

recall@5	[0.038146],	||	 recall@20	[0.109485],	||	 recall@50	[0.190705]
precision@5	[0.278576],	||	 precision@20	[0.213055],	||	 precision@50	[0.157652]
